In [20]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from datetime import  timedelta, timedelta, datetime

# Extração de dados API

## Lista de DAGS

In [21]:
baseUrl = "http://localhost:8080/api/v1"

username = 'airflow'
password = '1234'

In [22]:
try:
  response = requests.get(baseUrl + "/dags?only_active=true&tags=ativo", auth=HTTPBasicAuth(username, password))
  if response.status_code == 200:
    print("sucesso!")
    data = response.json()
    dagsDF = pd.DataFrame(data['dags'])
    dagsDF = dagsDF[['dag_id', 'dag_display_name', 'is_active', 'is_paused', 'is_subdag', 'root_dag_id', 'owners', 'tags', 'schedule_interval', 'timetable_description']]
except Exception as e:
  print(e)

sucesso!


In [23]:
dagsDF.head()

,dag_id,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,earthquake_data_upstream,Upstream EarthQuake Data,True,False,False,None,[Area 1],"[{'name': 'prd'}, {'name': 'ativo'}]",None,"Never, external triggers only"
1,processa_dados_EarthQuake,Data Count,True,False,False,None,[Area 1],"[{'name': 'prd'}, {'name': 'ativo'}]",None,"Never, external triggers only"
2,teste_execucao,Teste de Execucao,True,False,False,None,[Tester],"[{'name': 'test'}, {'name': 'ativo'}]",None,"Never, external triggers only"
3,teste_falha,Teste de Falha,True,False,False,None,[Tester],"[{'name': 'test'}, {'name': 'ativo'}]",None,"Never, external triggers only"


## DAGS em Execucao...

In [24]:
today = datetime.now()
yesterday = today - timedelta(days=1)
tomorrow = today + timedelta(days=1)
print(yesterday, tomorrow)

2024-10-24 14:27:40.827992 2024-10-26 14:27:40.827992


In [25]:
payload = {
    # "dag_ids": [
    #     "processa_dados_EarthQuake",
    #     "earthquake_data_upstream"
    # ],
    "states": [
        "failed",
        "running",
        "queued",
        "success"
    ],
    "start_date_gte": yesterday.strftime("%Y-%m-%dT00:00:00.000Z"),
    "start_date_lte": tomorrow.strftime("%Y-%m-%dT00:00:00.000Z")
}

In [26]:
try:
  response = requests.post(baseUrl + "/dags/~/dagRuns/list", json=payload, auth=HTTPBasicAuth(username, password))
  if response.status_code == 200:
    print("sucesso!")
    data = response.json()
    runsDF = pd.DataFrame(data['dag_runs'])
    runsDF = runsDF[['dag_id','start_date', 'end_date', 'run_type', 'state']]
    
    runsDF['start_date'] = pd.to_datetime(runsDF['start_date'])
    runsDF['end_date'] = pd.to_datetime(runsDF['end_date'])
    runsDF['start_hour'] = runsDF['start_date'].dt.hour
    runsDF['execution_time'] = (runsDF['end_date'] - runsDF['start_date']).dt.total_seconds() / 60
    
    runsDF['start_date'] = runsDF['start_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    runsDF['end_date'] = runsDF['end_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    runsDF['execution_date'] = runsDF['execution_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    
except Exception as e:
  print(e)

runsDF.head()

sucesso!
'execution_date'


,dag_id,start_date,end_date,run_type,state,start_hour,execution_time
0,teste_falha,25-10-2024 17:26:08,NaN,manual,running,17,NaN
1,earthquake_data_upstream,25-10-2024 17:26:08,25-10-2024 17:27:13,manual,success,17,1.079887
2,teste_execucao,25-10-2024 17:26:08,NaN,manual,running,17,NaN
3,processa_dados_EarthQuake,25-10-2024 17:26:10,25-10-2024 17:27:07,manual,success,17,0.949545


## Join df

In [15]:
df = pd.merge(runsDF, dagsDF, on='dag_id', how='inner')
df['tags'] = df['tags'].apply(lambda x: ', '.join([item['name'] for item in x]))
df['owners'] = df['owners'].apply(lambda x: x[0] if x else '')

df.head()

KeyError: 'dag_id'

In [31]:
df.to_csv('./dados_modelo.csv', index=False, header=True)

# Dados extraidos

In [18]:
df = pd.read_csv('./dados_modelo.csv', sep=';', encoding="latin1")
df.head()

,dag_id,start_date,end_date,run_type,state,start_hour,execution_time,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,teste_falha,22/10/2024 14:03,22/10/2024 14:10,manual,failed,14,7.0,Teste de Falha,True,False,False,NaN,Tester,"test, ativo",NaN,"Never, external triggers only"
1,novo_teste,24/10/2024 10:00,24/10/2024 10:05,manual,success,10,5.0,Novo Teste,True,False,False,NaN,Equipe de Teste,"test, ativo",NaN,"Never, external triggers only"
2,dados_clima,25/10/2024 08:00,NaN,scheduled,running,8,NaN,Dados Climáticos,True,False,False,NaN,Area 2,"prd, ativo",NaN,"Never, external triggers only"
3,monitoramento,26/10/2024 12:00,26/10/2024 12:05,manual,success,12,5.0,Monitoramento,True,False,False,NaN,Equipe de Monitoramento,"mon, ativo",NaN,"Never, external triggers only"
4,dados_financas,27/10/2024 09:00,27/10/2024 09:10,scheduled,success,9,10.0,Dados Financeiros,True,False,False,NaN,Area 3,"prd, ativo",NaN,"Never, external triggers only"


In [19]:
df = pd.read_json('./airflow_data.json')
df.head()

,dag_id,start_date,end_date,run_type,state,start_hour,execution_time,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,processamento_diario,25/10/2024 01:00,25/10/2024 01:15,scheduled,success,1,15,Processamento Diário,True,False,False,,Equipe de Operações,"diario, ativo",0 1 * * *,Executa diariamente às 01:00
1,relatorio_financeiro,24/10/2024 22:00,24/10/2024 22:30,manual,failed,22,30,Relatório Financeiro,True,False,False,,Financeiro,"relatorio, financeiro",None,Executa manualmente
2,backup_semanal,23/10/2024 03:00,23/10/2024 04:45,scheduled,success,3,105,Backup Semanal,True,False,False,,Equipe de Infraestrutura,"backup, semanal",0 3 * * 0,Executa semanalmente aos domingos às 03:00
3,processamento_diario,24/10/2024 01:00,24/10/2024 01:15,scheduled,success,1,15,Processamento Diário,True,False,False,,Equipe de Operações,"diario, ativo",0 1 * * *,Executa diariamente às 01:00
4,processamento_semanal,21/10/2024 02:00,21/10/2024 02:30,scheduled,success,2,30,Processamento Semanal,True,False,False,,Equipe de Operações,"semanal, ativo",0 2 * * 0,Executa semanalmente às 02:00


In [5]:
df['hora'] = df['start_date'].dt.hour
df['hora_arredondada'] = df['data_hora'].dt.round('H')

AttributeError: Can only use .dt accessor with datetimelike values